In [105]:
import pandas as pd
import numpy as np

import keras as keras
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model

In [3]:
#import CCLE data
E_CCLE = pd.read_csv('E_CCLE.csv', index_col=0)
M_CCLE = pd.read_csv('M_CCLE.csv',index_col=0)


In [4]:
E_TCGA = pd.read_csv('E_TCGA.csv',index_col=0)

In [5]:
M_TCGA = pd.read_csv('M_TCGA.csv',index_col=0)

In [6]:
IC50 = pd.read_csv('IC50_CCLE_author.csv',index_col=0)

In [7]:
E_CCLE = E_CCLE.transpose()
E_CCLE.head()

,ETS1,CATIP,MIR34B,ADAMTS8,LINC01124,ZBTB20-AS4,LRP1B,MOGS,HOXC-AS3,OTOG,...,OBSCN,ALAS1,SCARNA8,MLST8,LACTB2,NFASC,PRR27,TBL1XR1,OR51G1,GRAMD1C
NIHOVCAR3,4.055716,0.263034,0.0,2.104337,0.250962,0.163499,0.871844,5.570159,0.641546,0.344828,...,3.874797,5.246408,0.000000,5.523248,5.481557,2.963474,0.014355,5.813012,0.0,2.786596
HL60,0.739848,0.042644,0.0,0.028569,0.000000,0.000000,0.000000,5.204376,0.000000,0.000000,...,0.137504,6.418359,0.000000,6.131343,3.266037,0.189034,0.000000,4.297925,0.0,0.014355
CACO2,3.117695,0.111031,0.0,0.042644,0.831877,0.000000,0.042644,5.580447,0.056584,0.286881,...,1.176323,5.414812,0.000000,5.115200,6.356672,0.378512,0.000000,3.730096,0.0,1.280956
HEL,7.598574,0.565597,0.0,0.056584,0.189034,0.000000,0.014355,5.250583,0.000000,0.176323,...,4.122673,5.782933,2.990955,6.146289,5.060047,0.111031,0.000000,4.605850,0.0,1.021480
HEL9217,8.145728,0.298658,0.0,0.014355,0.097611,0.070389,0.000000,4.954196,0.014355,0.000000,...,2.954196,5.481234,0.000000,5.722466,5.233811,0.124328,0.000000,5.445594,0.0,0.887525


In [8]:
M_CCLE = M_CCLE.transpose()
M_CCLE.head()

,TBX19,TMCO6,MKI67,CDH6,TRPC1,ETS1,CLSTN3,RGPD6,PLEKHN1,KRTAP20-1,...,FBXO28,SPRR2F,CD300LG,CYB5A,CYP4F2,TBL1XR1,PNRC2,KCNE2,SF3A2,GRAMD1C
201T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22RV1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2313287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
253J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253JBV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
M_TCGA = M_TCGA.transpose()
M_TCGA.head()

,TBX19,TMCO6,MKI67,CDH6,TRPC1,ETS1,CLSTN3,RGPD6,PLEKHN1,KRTAP20-1,...,FBXO28,SPRR2F,CD300LG,CYB5A,CYP4F2,TBL1XR1,PNRC2,KCNE2,SF3A2,GRAMD1C
0004d251-3f70-4395-b175-c94c2f5b1b81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000d566c-96c7-4f1c-b36e-fa2222467983,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0011a67b-1ba9-4a32-a6b8-7850759a38cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001887aa-36d0-463f-8bca-dec7043b4f2e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001944e5-af34-4061-9c09-bb9ea346f6fd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
E_TCGA = E_TCGA.transpose()
E_TCGA.head()

,ETS1,CATIP,MIR34B,ADAMTS8,LINC01124,ZBTB20-AS4,LRP1B,MOGS,HOXC-AS3,OTOG,...,OBSCN,ALAS1,SCARNA8,MLST8,LACTB2,NFASC,PRR27,TBL1XR1,OR51G1,GRAMD1C
0004d251-3f70-4395-b175-c94c2f5b1b81,16.027345,11.869594,-9.965784,7.584970,15.447471,-9.965784,-9.965784,17.577340,-9.965784,15.050699,...,10.857204,20.859636,-9.965784,16.890003,16.178120,16.362406,10.007029,17.149013,-9.965784,12.691308
000d566c-96c7-4f1c-b36e-fa2222467983,17.984969,13.958462,-9.965784,14.822123,11.078151,7.033434,10.751545,18.721833,-9.965784,5.285439,...,17.372627,17.944398,-9.965784,18.279923,15.043198,16.044458,7.257397,18.837905,-9.965784,14.983216
0011a67b-1ba9-4a32-a6b8-7850759a38cf,19.086699,9.744836,-9.965784,11.464546,12.231522,6.584978,6.857993,18.448399,1.585443,8.189829,...,15.780232,18.232641,5.554620,17.427968,17.220416,16.469769,-9.965784,19.250266,-9.965784,14.008429
001887aa-36d0-463f-8bca-dec7043b4f2e,17.034885,13.605248,-9.965784,10.784636,16.775842,6.569871,5.977303,18.404248,4.322000,6.658226,...,11.806953,21.763549,-9.965784,17.920615,18.067566,14.661110,10.832891,18.463832,-9.965784,15.740176
001944e5-af34-4061-9c09-bb9ea346f6fd,18.023949,10.115045,-9.965784,12.508290,-9.965784,5.584993,12.330917,18.694909,-9.965784,5.044438,...,14.592749,17.939281,6.584978,18.493242,17.248863,12.841171,-9.965784,20.048447,4.322000,15.648723


In [11]:
IC50.head()

,X22RV1,X2313287,X42MGBA,X5637,X639V,X647V,X697,X769P,X786O,X8305C,...,VCAP,VMCUB1,VMRCRCW,VMRCRCZ,WM115,WSUDLCL2,YAPC,YH13,YKG1,ZR7530
TL-2-105,4.6782,4.7229,4.1668,3.71110,4.4224,4.6946,1.19990,4.989000,4.1650,5.4715,...,4.2607,4.8047,4.53320,3.787200,3.5018,2.5862,5.5113,4.0055,4.61160,5.714937
TAK-715,4.7217,3.7747,5.5256,3.73420,4.9266,3.9165,1.60470,2.609200,3.1698,6.0290,...,4.6174,5.4785,4.14860,3.748600,5.1913,2.8146,5.7889,5.5701,3.70750,5.853699
CP466722,2.1995,2.9851,3.5194,1.80130,3.2444,2.0760,-0.86400,1.877600,2.1493,3.7663,...,1.8166,3.9553,2.80750,1.950200,1.9625,2.0836,3.6385,2.2756,2.41570,4.580076
BMS-345541,2.7886,3.4029,4.3641,2.67680,4.6885,3.3922,0.79615,1.491900,1.5246,5.1374,...,3.0436,4.6465,3.14600,3.087400,3.0976,2.4709,3.7157,3.5809,2.42940,4.263374
Genentech_Cpd_10,2.2161,2.8008,3.5257,-0.42507,2.1349,3.3500,-1.41520,-0.013295,-1.5783,5.3360,...,1.7366,2.5529,0.87061,-0.008367,1.8015,2.4950,5.6822,2.8236,0.91206,4.259721


In [12]:
#Train a gene expresion autoencoder with TCGA data (11284, 22374), Eenc (15,363, 1,024, 256, and 64)

In [50]:
encoding_dim = 64  # 64 floats -> compression of factor 343, assuming the input is 22374 floats

tumor_dim = E_TCGA.shape[1]
tumor_dim


22374

In [55]:
from keras import backend as K


In [149]:
K.clear_session()
layers = [1024,256,128,encoding_dim]
# this is our input placeholder
input_tumor = Input(shape=(tumor_dim,))
# "encoded" is the encoded representation of the input
for i,layer in enumerate(layers):
    encoded = Dense(layer, activation='relu')(input_tumor if i == 0 else encoded)
    encoded = BatchNormalization()(encoded)
for i,layer in enumerate(layers[-2::-1]):
    decoded = Dense(layer, activation='relu')(encoded if i == 0 else decoded)
    decoded = BatchNormalization()(decoded)
out = Dense(tumor_dim, activation='linear')(decoded)


In [142]:
layers[-2::-1]

[128, 256, 1024]

In [150]:
# this model maps an input to its reconstruction
autoencoder = Model(input_tumor, out)
# this model maps an input to its encoded representation
encoder = Model(input_tumor, encoded)

In [151]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 22374)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              22912000  
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
__________

In [152]:
# create a placeholder for an encoded (64-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = Model(encoded_input, decoder_layer(encoded_input))
#decoder = Model(encoded_input, decoder_layer(encoded_input))

In [217]:
adam = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
autoencoder.compile(optimizer= adam, loss='mean_squared_error')
#autoencoder.compile(optimizer= adam, loss='binary_crossentropy')


In [204]:
len(list(E_TCGA.index.values))

11284

In [205]:
#E_TCGA_norm = (E_TCGA-E_TCGA.mean())/E_TCGA.std() #Mean - 0 Std_dev-  1 
E_TCGA_norm = (E_TCGA - E_TCGA.min()) / (E_TCGA.max() - E_TCGA.min()) #Values between 0 and 1

In [206]:
E_TCGA_norm['ALAS1'].mean(), E_TCGA_norm['ALAS1'].std()

(0.47912599969696307, 0.09160637255628065)

In [207]:
E_TCGA_norm['ALAS1'].describe()

count    11284.000000
mean         0.479126
std          0.091606
min          0.000000
25%          0.424869
50%          0.475984
75%          0.530270
max          1.000000
Name: ALAS1, dtype: float64

In [208]:
E_TCGA['ALAS1'].describe()

count    11284.000000
mean        18.155262
std          1.046437
min         12.682117
25%         17.535477
50%         18.119369
75%         18.739491
max         24.105303
Name: ALAS1, dtype: float64

In [209]:
#reserve a random sample of 2000 samples to validate the model
from random import sample 
valid_idx = sample(list(E_TCGA_norm.index.values),2000 )
x_valid =  E_TCGA_norm.loc[valid_idx, :]
x_train = E_TCGA_norm.drop(valid_idx, axis=0)
print (x_train.shape) 
print (x_valid.shape)

(9284, 22374)
(2000, 22374)


In [218]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_valid, x_valid))

Train on 9284 samples, validate on 2000 samples
Epoch 1/50
9284/9284 [==============================] - 19s 2ms/step - loss: 0.0487 - val_loss: 0.0231
Epoch 2/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0211 - val_loss: 0.0211
Epoch 3/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0206 - val_loss: 0.0206
Epoch 4/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0205 - val_loss: 0.0206
Epoch 5/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0205 - val_loss: 0.0208
Epoch 6/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0205 - val_loss: 0.0216
Epoch 7/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0205 - val_loss: 0.0219
Epoch 8/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.0206 - val_loss: 0.0212
Epoch 9/50
4352/9284 [=============>................] - ETA: 8s - loss: 0.0206

KeyboardInterrupt: 

In [219]:
p = autoencoder.predict(x_valid)
p[0][:100]

array([0.5755074 , 0.7250396 , 0.06145394, 0.7029169 , 0.6552242 ,
       0.5738788 , 0.6149176 , 0.7126174 , 0.64587486, 0.506714  ,
       0.6896518 , 0.09259804, 0.5394136 , 0.63534915, 0.5596074 ,
       0.6940607 , 0.52941775, 0.55254835, 0.6092564 , 0.6004694 ,
       0.49767038, 0.4442839 , 0.06485688, 0.3554048 , 0.3469394 ,
       0.8071205 , 0.62524134, 0.7459519 , 0.75308704, 0.7086085 ,
       0.6224297 , 0.7472969 , 0.8545271 , 0.6790348 , 0.7054677 ,
       0.5232011 , 0.7857597 , 0.7086949 , 0.7355695 , 0.7347603 ,
       0.0009932 , 0.59844995, 0.6468655 , 0.71287864, 0.48272896,
       0.6870252 , 0.7255891 , 0.5714047 , 0.7487041 , 0.7531857 ,
       0.5806849 , 0.5266148 , 0.84900904, 0.77229464, 0.7706752 ,
       0.7313118 , 0.6671402 , 0.5481683 , 0.6915494 , 0.78988683,
       0.48053285, 0.7647283 , 0.09809243, 0.661294  , 0.7084047 ,
       0.76405543, 0.06726572, 0.0325187 , 0.80566126, 0.59240913,
       0.0169968 , 0.44141158, 0.7822436 , 0.7721879 , 0.59465

In [220]:
x_valid.iloc[0,:100].values

array([0.62032507, 0.73491709, 0.        , 0.68263655, 0.8495766 ,
       0.65840216, 0.60692686, 0.67225922, 0.        , 0.58748526,
       0.64546019, 0.        , 0.44477353, 0.48559713, 0.56937685,
       0.66391336, 0.68820836, 0.52326942, 0.49239311, 0.57072383,
       0.50578852, 0.63113802, 0.60327572, 0.76808253, 0.43940671,
       0.79610674, 0.66769065, 0.79540338, 0.71235117, 0.73239666,
       0.59592646, 0.79058943, 0.84457563, 0.69775536, 0.61147024,
       0.8254938 , 0.77831311, 0.66487692, 0.7532442 , 0.73059528,
       0.        , 0.55616082, 0.80239987, 0.70303012, 0.53110623,
       0.66166919, 0.65584961, 0.56753771, 0.8339753 , 0.72500315,
       0.64647812, 0.55133374, 0.86249923, 0.84493017, 0.73804831,
       0.73175178, 0.6276272 , 0.55820447, 0.68041466, 0.76595438,
       0.67247524, 0.79686048, 0.        , 0.62953315, 0.6094391 ,
       0.66060232, 0.        , 0.        , 0.7396596 , 0.45310157,
       0.        , 0.4197819 , 0.89220681, 0.75852596, 0.53693

In [221]:
#MSE on validation
substract = (p - x_valid)
np.sqrt(np.power(substract.values, 2)).mean()

0.09182654007973066